In [1]:
# import dependencies
# import sys
# !{sys.executable} -m pip install --user install tensorflow==2.8.0. 

from PIL import Image
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import glacierml as gl
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.cluster import KMeans
import seaborn as sns
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_columns', None)

tf.random.set_seed(42)

print('currently running tensorflow version: ' + tf.__version__)
# RGI.to_csv('Farinotti_mean_thickness_RGI_ID.csv')


currently running tensorflow version: 2.8.0


In [ ]:
predictions = gl.predictions_finder()
predictions = predictions.reset_index()
predictions = predictions.drop('index', axis = 1)

df = pd.DataFrame(columns = {
        'RGIId','0', '1', '2', '3', '4', '5', '6', '7', '8', '9','10',
        '11','12','13','14','15','16','17','18','19','20','21',
        '22','23','24',
})

for index in tqdm(predictions.index):
    idx = index
#     print(idx)

    training_module =  predictions['coregistration'].iloc[idx]
    architecture = predictions['architecture'].iloc[idx]
    learning_rate = predictions['learning rate'].iloc[idx]
    epochs = '2000'
    df_glob = gl.global_predictions_loader(
        training_module = training_module,
        architecture = architecture,
        learning_rate = learning_rate,
        epochs = epochs

    )
    

    df = pd.concat([df,df_glob])

df = df[[
        'RGIId','0', '1', '2', '3', '4', '5', '6', '7', '8', '9','10',
        '11','12','13','14','15','16','17','18','19','20','21',
        '22','23','24',
]]

compiled_raw = df.groupby('RGIId')[
        '0', '1', '2', '3', '4', '5', '6', '7', '8', '9','10',
        '11','12','13','14','15','16','17','18','19','20','21',
        '22','23','24',
]

dft = pd.DataFrame()
for this_rgi_id, obj in tqdm(compiled_raw):
    rgi_id = pd.Series(this_rgi_id, name = 'RGIId')
#     print(f"Data associated with RGI_ID = {this_rgi_id}:")
#     print(this_rgi_id)
#     break
    dft = pd.concat([dft, rgi_id])
    dft = dft.reset_index()
    dft = dft.drop('index', axis = 1)
    dft.loc[dft.index[-1], 'Mean Thickness'] = obj[[
        '0', '1', '2', '3', '4', '5', '6', '7', '8', '9','10',
        '11','12','13','14','15','16','17','18','19','20','21',
        '22','23','24',
    ]].mean().mean()
    dft.loc[dft.index[-1],'Thickness Std Dev'] = obj[[
        '0', '1', '2', '3', '4', '5', '6', '7', '8', '9','10',
        '11','12','13','14','15','16','17','18','19','20','21',
        '22','23','24',
    ]].stack().std()
    
dft = dft.rename(columns = {
    0:'RGIId'
})
dft

100%|██████████| 161/161 [12:23<00:00,  4.62s/it]
/home/sa42/miniconda3/envs/python-cartopy-f/lib/python3.7/site-packages/ipykernel_launcher.py:37: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
 78%|███████▊  | 168583/216459 [19:05<08:08, 98.00it/s] 

In [10]:
dft = pd.read_csv('aggregated/sermeq_aggregated_bootstrap_predictions.csv')
ref = pd.read_csv('reference_thicknesses/Farinotti_mean_thickness_RGI_ID_2.csv')
ref = ref[[
    'RGIId',
    'Farinotti Mean Thickness'
]]
ref['region'] = ref['RGIId'].str[6:8]
ref = ref.sort_values('RGIId')
ref = ref.dropna()
RGI = gl.RGI_loader()
RGI = RGI[[
    'RGIId',
    'CenLat',
    'CenLon',
    'Slope',
    'Zmin',
    'Zmed',
    'Zmax',
    'Area',
    'Aspect',
    'Lmax'
]]

# drops = RGI[
# #     (RGI['Aspect'] < 0) | 
#     (RGI['Zmin'] < 0) |
#     (RGI['Zmax'] < 0) |
#     (RGI['Lmax'] < 0) 

# ].index
# RGI = RGI.drop(drops)
RGI['Zdelta'] = RGI['Zmax'] - RGI['Zmin']
# RGI

ref = pd.merge(ref, RGI, on = 'RGIId')

ref = pd.merge(ref, dft, 
#                left_index = True, right_index = True)
on = [
    'RGIId'
])
ref = ref.rename(columns = {
    'Mean Thickness':'Edasi Mean Thickness'
})

ref['Farinotti Volume'] = (ref['Farinotti Mean Thickness'] / 1e3 )* ref['Area']

ref['region'] = ref['RGIId'].str[6:8]
ref['Edasi Volume'] = (ref['Edasi Mean Thickness'] / 1e3) * ref['Area']
ref['Volume Std Dev'] = (ref['Thickness Std Dev'] / 1e3 )* ref['Area']
ref = ref.reset_index()
ref = ref.drop('index', axis = 1)

ref['VE / VF'] = ref['Edasi Mean Thickness'] / ref['Farinotti Mean Thickness']
ref = ref.drop_duplicates()
ref = ref.dropna()
# sum(ref['volume km3'])
ref

,RGIId,Farinotti Mean Thickness,region,CenLat,CenLon,Slope,Zmin,Zmed,Zmax,Area,Aspect,Lmax,Zdelta,Unnamed: 0,Edasi Mean Thickness,Median Thickness,Thickness Std Dev,Shapiro-Wilk statistic,Shapiro-Wilk p_value,IQR,Lower Bound,Upper Bound,Median Value,Farinotti Volume,Edasi Volume,Volume Std Dev,VE / VF
0,RGI60-01.00001,21.182035,01,63.6890,-146.8230,42.0,1936,2385,2725,0.360,346,839,789,0,14.841929,14.226050,7.197743,0.974259,1.721300e-27,8.942386,7.885554,21.626253,14.226050,0.007626,0.005343,0.002591,0.700685
1,RGI60-01.00002,30.519478,01,63.4040,-146.6680,16.0,1713,2005,2144,0.558,162,1197,431,1,35.318791,35.053523,5.178314,0.966729,9.273571e-31,5.944721,30.864175,39.845612,35.053523,0.017030,0.019708,0.002889,1.157254
2,RGI60-01.00003,35.518610,01,63.3760,-146.0800,18.0,1609,1868,2182,1.685,175,2106,573,2,33.070411,32.592387,4.846715,0.958427,9.373772e-34,5.443084,28.916565,37.291615,32.592387,0.059849,0.055724,0.008167,0.931073
3,RGI60-01.00004,53.077263,01,63.3810,-146.1200,19.0,1273,1944,2317,3.681,195,4175,1044,3,40.322059,39.819495,5.175665,0.980968,6.835554e-24,6.095077,35.648320,45.116479,39.819495,0.195377,0.148425,0.019052,0.759686
4,RGI60-01.00005,47.519207,01,63.5510,-147.0570,16.0,1494,1914,2317,2.573,181,2981,823,4,42.108854,41.651334,5.433712,0.983602,3.243421e-22,6.543149,37.342863,47.168816,41.651334,0.122267,0.108346,0.013981,0.886144
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215482,RGI60-19.02748,12.948014,19,-53.9860,-37.7325,29.9,310,-999,510,0.042,315,255,200,216454,26.798332,24.811207,13.271253,0.961854,1.399892e-32,17.077573,14.143986,39.744851,24.811207,0.000544,0.001126,0.000557,2.069687
215483,RGI60-19.02749,22.894110,19,-54.8310,-36.1361,23.6,330,-999,830,0.567,200,1130,500,216455,37.231512,35.736298,14.775759,0.980468,3.438035e-24,19.285031,23.007389,51.508082,35.736298,0.012981,0.021110,0.008378,1.626249
215484,RGI60-19.02750,60.874610,19,-54.1884,-37.3018,16.8,10,-999,1110,4.118,308,4329,1100,216456,73.363529,71.971468,21.894920,0.992775,3.735962e-14,28.705928,52.045433,94.994497,71.971468,0.250682,0.302111,0.090163,1.205158
215485,RGI60-19.02751,10.053704,19,-68.8656,-90.4266,0.4,170,-999,270,0.011,122,106,100,216457,90.873023,88.635585,25.404969,0.988782,3.084626e-18,33.489690,66.231848,116.174542,88.635585,0.000111,0.001000,0.000279,9.038761


In [ ]:
predictions = gl.predictions_finder()
predictions = predictions.reset_index()
predictions = predictions.drop('index', axis = 1)
predictions.sort_values('predicted volume')

In [ ]:
predictions[predictions['architecture'] == '16-10']

In [ ]:
idx = 122
training_module =  predictions['coregistration'].iloc[idx]
architecture = '_' + predictions['architecture'].iloc[idx]
learning_rate = predictions['learning rate'].iloc[idx]
epochs = '2000'
df_glob = gl.global_predictions_loader(
    training_module = training_module,
    architecture = architecture,
    learning_rate = learning_rate,
    epochs = epochs
    
)
df_reg = gl.regional_predictions_loader(
    training_module = training_module,
    architecture = architecture,
    learning_rate = learning_rate,
    epochs = epochs
    
)

gl.glathida_stats_adder(
    df_reg,
#     pth_1 = '/home/simonhans/data/prethicktor/regional_data/raw/',
#     pth_2 = '/home/simonhans/data/prethicktor/RGI/rgi60-attribs/',
#     pth_3 = '/home/simonhans/data/prethicktor/regional_data/training_data/',
)
df_reg

df_glob
df = df_glob[[
    'avg predicted thickness','0', '1', '2', '3', '4', '5', '6', '7', '8', '9','10',
    '11','12','13','14','15','16','17','18','19','20','21',
    '22','23','24',
]]

In [ ]:
ref = pd.read_csv('reference_thicknesses/Farinotti_mean_thickness_RGI_ID_2.csv')
ref = ref[[
    'RGIId',
    'Farinotti Mean Thickness'
]]
ref['region'] = ref['RGIId'].str[6:8]
ref = ref.sort_values('RGIId')
ref = ref.dropna()

RGI = gl.RGI_loader()
RGI = RGI[[
    'RGIId',
    'CenLat',
    'CenLon',
    'Slope',
    'Zmin',
    'Zmed',
    'Zmax',
    'Area',
    'Aspect',
    'Lmax'
]]
# drops = RGI[

# #     (RGI['Aspect'] < 0) | 
#     (RGI['Zmin'] < 0) |
#     (RGI['Zmax'] < 0) |
#     (RGI['Lmax'] < 0) 

# ].index
# RGI = RGI.drop(drops)
RGI['Zdelta'] = RGI['Zmax'] - RGI['Zmin']
# RGI

ref = pd.merge(ref, RGI, on = 'RGIId')

ref = pd.merge(ref, dft, on = 'RGIId')
ref = ref.rename(columns = {
    'Mean Thickness':'Edasi Mean Thickness'
})
ref = ref.reset_index()
ref = ref.drop('index', axis = 1)
ref['VE / VF'] = ref['Edasi Mean Thickness'] / ref['Farinotti Mean Thickness']
ref

In [ ]:
fig = plt.figure(figsize=(15, 12))
plt.subplots_adjust(hspace=0.5)
plt.suptitle('Residuals by RGI feature' + 
    ',\nLayer Architecture: ' + predictions['architecture'].loc[idx] +
    ', Learning Rate: ' + predictions['learning rate'].loc[idx], fontsize=18, y=0.99)
fig.patch.set_facecolor('w')
dft = ref[[
    'CenLat',
    'CenLon',
    'Slope',
    'Zmin',
    'Zmed',
    'Zmax',
    'Zdelta',
    'Area',
    'Aspect',
    'Lmax',

]]
for n, variable in enumerate(dft):
    ax = plt.subplot(5, 2, n + 1)
    plt.subplots_adjust(hspace=0.5)
    ax.set_ylabel('VE / VF')
    ax.set_xlabel(variable)
    ax.set_yscale('log')
    plt.scatter(
        dft[variable],
        ref['VE / VF'],
        marker = '.'
    )
    ax.fill_between(
        x = (
            dft[variable].min(), dft[variable].max()
        ),
        y1 = 1.25,
        y2 = 0.75,
        facecolor = 'gray',
        alpha = 0.2
    )
    plt.axhline(y = 1.00, color = 'red', linestyle = '--')

plt.tight_layout()

In [ ]:
fig = plt.figure(figsize=(15, 12))
plt.subplots_adjust(hspace=0.5)
plt.suptitle('Coefficient of Variation by RGI feature' + 
    ',\nLayer Architecture: ' + predictions['architecture'].loc[idx] +
    ', Learning Rate: ' + predictions['learning rate'].loc[idx], fontsize=18, y=0.99)
fig.patch.set_facecolor('w')
dft = ref[[
    'CenLat',
    'CenLon',
    'Slope',
    'Zmin',
    'Zmed',
    'Zmax',
    'Zdelta',
    'Area',
    'Aspect',
    'Lmax',

]]

for n, variable in enumerate(dft):

    ax = plt.subplot(5, 2, n + 1)
    plt.subplots_adjust(hspace=0.5)
    ax.set_ylabel('CV')
    ax.set_xlabel(variable)
    ax.set_yscale('log')
    plt.scatter(
        dft[variable],
        (ref[['0', '1', '2', '3','4','5','6','7','8','9','10',
          '11','12','13','14','15','16','17','18','19','20',
          '21','22','23','24']].std(axis = 1)) / ref[['0', '1', '2', '3','4','5','6','7','8','9','10',
          '11','12','13','14','15','16','17','18','19','20',
          '21','22','23','24']].mean(axis = 1) ,
        marker = '.'
    )
    ax.fill_between(
        x = (
            dft[variable].min(), dft[variable].max()
        ),
        y1 = 1.25,
        y2 = 0.75,
        facecolor = 'gray',
        alpha = 0.2
    )
    plt.axhline(y = 1.00, color = 'red', linestyle = '--')

plt.tight_layout()

In [ ]:
data_1 = ref['Farinotti Mean Thickness'] * ref['Area']
data_2 = ref['Edasi Mean Thickness'] * ref['Area']
#sort data


x_1 = np.sort(data_1)
# x = data_1
#calculate CDF values
y_1 = 1. * np.arange(len(data_1)) / (len(data_1) - 1)


x_2 = np.sort(data_2)
# x = data
#calculate CDF values
y_2 = 1. * np.arange(len(data_2)) / (len(data_2) - 1)

#plot CDF
fig, ax = plt.subplots(1,1,figsize=(8, 8))

plt.subplots_adjust(hspace=0.5)

plt.suptitle('RGI Volume Cumulative Distribution Function', fontsize=18, y=0.93)
fig.patch.set_facecolor('w')


plt.subplots_adjust(hspace=0.5)
plt.plot(
    x_1, 
    y_1,
    c = 'blue',
    label = 'Farinotti Volume'
    
)
plt.plot(
    x_2, 
    y_2,
    c = 'orange',
    label = 'Edasi Volume'
)
plt.legend()
ax.set_xscale('log')
ax.set_xlabel('Ice Volume')
ax.set_ylabel('% of Glaciers Containing Ice')
plt.savefig('figs/cdf/cdf.svg')

In [ ]:
dfr = pd.DataFrame()
for i in range(0, 90, 10):
    dft = ref.loc[(ref['Slope'].between(i, i + 10)) & (ref['VE / VF'] <= 10)]
    dft['Slope (degrees)'] = str(i) + '-' + str(i + 10)
    dfr = pd.concat([dfr, dft])
ax = sns.violinplot(
    x = dfr['Slope (degrees)'],
    y = dfr['VE / VF'],
    scale = 'count',
    inner = 'box'
#     ax = plt.subplot(1,1,(figsize = (10,10)))
)
plt.axhline(y = 1.00, color = 'red', linestyle = '--')
ax.set_title('V_Edasi / V_Farinotti against Slope')
print(len(dfr))
plt.savefig('figs/violin/slope.svg')

In [ ]:
dfr = pd.DataFrame()
for i in range(0, 4000, 500):
    dft = ref.loc[(ref['Area'].between(i, i + 500)) & (ref['VE / VF'] <= 10)]
    dft['Area (km2)'] = str(i) + '-' + str(i + 500)
    dfr = pd.concat([dfr, dft])
ax = sns.violinplot(
    x = dfr['Area (km2)'],
    y = dfr['VE / VF'],
    inner = 'box',
    scale = 'count',
#     inner = None
#     ax = plt.subplot(1,1,(figsize = (10,10)))
)
plt.axhline(y = 1.00, color = 'red', linestyle = '--')
ax.set_title('VE / VF against Area')
plt.savefig('figs/violin/area.svg')

In [ ]:
dfr = pd.DataFrame()
for i in range(0, 10000, 1000):
    dft = ref.loc[(ref['Lmax'].between(i, i + 1000)) & (ref['VE / VF'] <= 10)]
    dft['Lmax (m)'] = str(i) + '-' + str(i + 1000)
    dfr = pd.concat([dfr, dft])
ax = sns.violinplot(
    x = dfr['Lmax (m)'],
    y = dfr['VE / VF'],
    scale = 'count',
    inner = 'box'
#     ax = plt.subplot(1,1,(figsize = (10,10)))
)
plt.axhline(y = 1.00, color = 'red', linestyle = '--')
ax.set_title('VE / VF against Lmax (m)')

In [ ]:
dfr = pd.DataFrame()
for i in range(0, 6000, 1000):
    dft = ref.loc[(ref['Zmin'].between(i, i + 1000)) & (ref['VE / VF'] <= 10)]
    dft['Zmin (m)'] = str(i) + '-' + str(i + 1000)
    dfr = pd.concat([dfr, dft])
ax = sns.violinplot(
    x = dfr['Zmin (m)'],
    y = dfr['VE / VF'],
    scale = 'count',
    
#     ax = plt.subplot(1,1,(figsize = (10,10)))
)
plt.axhline(y = 1.00, color = 'red', linestyle = '--')
ax.set_title('VE / VF against Zmin')


In [ ]:
dfr = pd.DataFrame()
for i in range(0, 6000, 1000):
    dft = ref.loc[(ref['Zmed'].between(i, i + 1000)) & (ref['VE / VF'] <= 10)]
    dft['Zmed (m)'] = str(i) + '-' + str(i + 1000)
    dfr = pd.concat([dfr, dft])
ax = sns.violinplot(
    x = dfr['Zmed (m)'],
    y = dfr['VE / VF'],
    scale = 'count'
#     ax = plt.subplot(1,1,(figsize = (10,10)))
)
plt.axhline(y = 1.00, color = 'red', linestyle = '--')
ax.set_title('VE / VF against Zmed')

In [ ]:
dfr = pd.DataFrame()
for i in range(0, 6000, 1000):
    dft = ref.loc[(ref['Zmax'].between(i, i + 1000)) & (ref['VE / VF'] <= 10)]
    dft['Zmax (m)'] = str(i) + '-' + str(i + 1000)
    dfr = pd.concat([dfr, dft])
ax = sns.violinplot(
    x = dfr['Zmax (m)'],
    y = dfr['VE / VF'],
    scale = 'count'
#     ax = plt.subplot(1,1,(figsize = (10,10)))
)
plt.axhline(y = 1.00, color = 'red', linestyle = '--')
ax.set_title('VE / VF against Zmax')

In [ ]:
dfr = pd.DataFrame()
for i in range(0, 6000, 1000):
    dft = ref.loc[(ref['Zdelta'].between(i, i + 1000)) & (ref['VE / VF'] <= 10)]
    dft['Zdelta (m)'] = str(i) + '-' + str(i + 1000)
    dfr = pd.concat([dfr, dft])
ax = sns.violinplot(
    x = dfr['Zdelta (m)'],
    y = dfr['VE / VF'],
    scale = 'count'
#     ax = plt.subplot(1,1,(figsize = (10,10)))
)
plt.axhline(y = 1.00, color = 'red', linestyle = '--')
ax.set_title('VE / VF against Zdelta')


In [ ]:
dfr = pd.DataFrame()
for i in range(-90, 100, 10):
    dft = ref.loc[(ref['CenLat'].between(i, i + 10)) & (ref['VE / VF'] <= 10)]
    dft['Latitude (degrees)'] = str(i) + '-' + str(i + 10)
    dfr = pd.concat([dfr, dft])
ax = sns.violinplot(
    x = dfr['Latitude (degrees)'],
    y = dfr['VE / VF'],
    scale = 'count'
#     ax = plt.subplot(1,1,(figsize = (10,10)))
)
plt.axhline(y = 1.00, color = 'red', linestyle = '--')
ax.set_title('VE / VF against Cent Lat')


In [ ]:
dfr = pd.DataFrame()
for i in range(-180, 190, 50):
    dft = ref.loc[(ref['CenLon'].between(i, i + 50)) & (ref['VE / VF'] <= 10)]
    dft['Longitude (degrees)'] = str(i) + '-' + str(i + 50)
    dfr = pd.concat([dfr, dft])
ax = sns.violinplot(
    x = dfr['Longitude (degrees)'],
    y = dfr['VE / VF'],
    scale = 'count'
#     ax = plt.subplot(1,1,(figsize = (10,10)))
)
plt.axhline(y = 1.00, color = 'red', linestyle = '--')
ax.set_title('VE / VF against Cent Lon')